In [1]:
from glob import glob
import os
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
root = "/data/datasets/MSD"
for task in os.listdir(root):
    if task.startswith("."):
        continue
    
    for subset in ["labelsTr", "labelsTs"]:
        sample = []
        for gt_name in tqdm(os.listdir(os.path.join(root, task, subset)), desc=f"{task}_{subset}"):
            if gt_name.startswith("."):
                continue

            gt_path = os.path.join(task, subset, gt_name)
            gt_raw = nib.load(os.path.join(root, gt_path))
            gt = np.asanyarray(gt_raw.dataobj)
            
            for label in np.unique(gt):
                if label == 0:
                    continue
                
                pos = (gt == label).sum(axis=(0,1))
                n_pos = np.argwhere(pos).shape[0]
                sample.append((task, gt_path, label, n_pos))
                
        df = pd.DataFrame(sample, columns=["task", "gt_path", "obj_id", "n_pos"])
        print(df)
        # df.to_csv(f"./data/MSD/{task}_{subset}.csv")

Task10_Colon_labelsTr:   0%|          | 0/100 [00:00<?, ?it/s]

colon_166.nii.gz
colon_171.nii.gz
colon_129.nii.gz
colon_031.nii.gz
colon_176.nii.gz
colon_140.nii.gz
colon_192.nii.gz
colon_092.nii.gz
colon_007.nii.gz
colon_036.nii.gz
colon_161.nii.gz
colon_111.nii.gz
colon_159.nii.gz
colon_145.nii.gz
colon_131.nii.gz


KeyboardInterrupt: 